In [ ]:
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import RepeatedKFold, cross_validate

### Process Data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving proc.csv to proc.csv


In [ ]:
df = pd.read_csv('proc.csv')
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]

In [ ]:
df['cat_cluster'] = df['cat_cluster'].fillna(0) #Fill NAs with 0s (should be in above section but i forgot :/)
df['location_cluster'] = df['location_cluster'].fillna(0)

In [ ]:
str_cols = ['name', 'address', 'gps', 'top_category', 'relevant_cat', 'translated', 'charged_words', 'price_words', 'categories', 'unixReviewTime', 'reviewTime']
df_num = df[df.columns.drop(str_cols)]

In [ ]:
df_num['Class_$'].mean(),df_num['Class_$$'].mean(),df_num['Class_$$$'].mean(),df_num['Class_$$$$'].mean()

(0.0, 0.6347237880496054, 0.34949267192784667, 0.015783540022547914)

In [ ]:
df_num.shape

(887, 21)

In [ ]:
def cost_label_enc(row):
  #Still want to maintain ordinal values but no $ instances, so it doesn't make sense to encode values as [2, 3, 4]
  if row['Class_$$'] == 1:
    return 0
  elif row['Class_$$$'] == 1:
    return 1
  #For class $$$$:
  else:
    return 2

In [ ]:
df_num['cost'] =df_num.apply(cost_label_enc, axis=1)
df_num.drop(['Class_$', 'Class_$$', 'Class_$$$', 'Class_$$$$'], axis=1, inplace=True)

<ipython-input-10-a9b45a91013f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num['cost'] =df_num.apply(cost_label_enc, axis=1)
<ipython-input-10-a9b45a91013f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_num.drop(['Class_$', 'Class_$$', 'Class_$$$', 'Class_$$$$'], axis=1, inplace=True)


In [ ]:
X = df_num.drop(['cost'], axis=1).astype(float)
y = df_num[['cost']].astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print(y_train.value_counts())

cost
0       458
1       240
2        11
dtype: int64


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
#Scale all numbers
X_train_num = scaler.fit_transform(X_train)
X_test_num = scaler.fit_transform(X_test)

### XGBoost - No Oversampling

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

https://xgboost.readthedocs.io/en/stable/parameter.html

In [ ]:
xgb = XGBClassifier(
    n_estimators=2, max_depth=2,
    learning_rate=1,
    objective='multi:softmax',
    eval_metric='merror',
    #scale_pos_weight=
)

In [ ]:
xgb.fit(X_train_num, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='merror',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=2,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=2,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

In [ ]:
train_pred = xgb.predict(X_train_num)
print(classification_report(y_train, train_pred))

              precision    recall  f1-score   support

           0       0.68      0.92      0.78       458
           1       0.59      0.23      0.33       240
           2       0.00      0.00      0.00        11

    accuracy                           0.67       709
   macro avg       0.42      0.38      0.37       709
weighted avg       0.64      0.67      0.62       709



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pred = xgb.predict(X_test_num)

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.61      0.77      0.68       105
           1       0.48      0.31      0.38        70
           2       0.00      0.00      0.00         3

    accuracy                           0.58       178
   macro avg       0.36      0.36      0.35       178
weighted avg       0.55      0.58      0.55       178



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(confusion_matrix(y_test, pred))

[[81 24  0]
 [48 22  0]
 [ 3  0  0]]


### XGBoost - Oversampling

In [ ]:
import imblearn
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(
    sampling_strategy={
        0: 500,
        1: 300,
        2: 200
    }
)

X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

print(y_train_ros.value_counts())

cost
0       500
1       300
2       200
dtype: int64


/usr/local/lib/python3.10/dist-packages/imblearn/utils/_validation.py:313: UserWarning: After over-sampling, the number of samples (500) in class 0 will be larger than the number of samples in the majority class (class #0 -> 458)
  warnings.warn(


In [ ]:
y_train_ros = y_train_ros.astype(int)
y_test_ros = y_test.astype(int)

In [ ]:
X_train_num_ros = scaler.fit_transform(X_train_ros)
X_test_num_ros = scaler.fit_transform(X_test)

In [ ]:
xgb_oversampled = XGBClassifier(
    n_estimators=2, max_depth=2,
    learning_rate=1,
    objective='multi:softmax',
    eval_metric='merror'
)

In [ ]:
xgb_oversampled.fit(X_train_num_ros, y_train_ros)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='merror',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=2,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=2,
              n_jobs=None, num_parallel_tree=None, objective='multi:softmax', ...)

In [ ]:
oversampled_train_pred = xgb_oversampled.predict(X_train_num_ros)
print(classification_report(y_train_ros, oversampled_train_pred))

              precision    recall  f1-score   support

           0       0.57      0.91      0.70       500
           1       0.53      0.17      0.26       300
           2       0.87      0.42      0.57       200

    accuracy                           0.59      1000
   macro avg       0.66      0.50      0.51      1000
weighted avg       0.62      0.59      0.54      1000



In [ ]:
pred_oversampled = xgb_oversampled.predict(X_test_num_ros)

In [ ]:
print(classification_report(y_test_ros, pred_oversampled))

              precision    recall  f1-score   support

           0       0.63      0.85      0.72       105
           1       0.56      0.29      0.38        70
           2       0.00      0.00      0.00         3

    accuracy                           0.61       178
   macro avg       0.39      0.38      0.37       178
weighted avg       0.59      0.61      0.57       178



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(confusion_matrix(y_test_ros, pred_oversampled))

[[89 16  0]
 [50 20  0]
 [ 3  0  0]]


### Gridsearch - No oversampling

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, make_scorer

In [ ]:
xgb_gridsearch = XGBClassifier(objective='multi:softmax', eval_metric='merror')

In [ ]:
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.5, 1],
    'subsample': [0.6, 0.8, 1.0],
    'gamma': [0.0, 1.0],
    'colsample_bytree': [0.5, 1]
}

In [ ]:
grid_search = GridSearchCV(estimator=xgb_gridsearch, param_grid=param_grid,
                           scoring='f1_macro', cv=5, verbose=1)

In [ ]:
grid_search.fit(X_train_num, y_train)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='merror', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=N...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objective='multi:softmax', ...),
             param_grid={'colsample_bytree': [0.5, 1], 'gamma': [0.0, 1.0],
                         'learning_rate': [0.1, 0.5, 1], 'max_depth': [3, 4, 5],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='f1_macro', verbose=1)

In [ ]:
top_params = grid_search.best_params_
top_score = grid_search.best_score_
top_model = grid_search.best_estimator_

In [ ]:
top_params

{'colsample_bytree': 0.5,
 'gamma': 1.0,
 'learning_rate': 1,
 'max_depth': 5,
 'subsample': 0.6}

In [ ]:
top_score

0.34942860750949944

In [ ]:
gs_train_pred = xgb.predict(X_train_num)
print(classification_report(y_train, gs_train_pred))

              precision    recall  f1-score   support

           0       0.68      0.92      0.78       458
           1       0.59      0.23      0.33       240
           2       0.00      0.00      0.00        11

    accuracy                           0.67       709
   macro avg       0.42      0.38      0.37       709
weighted avg       0.64      0.67      0.62       709



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
top_pred = top_model.predict(X_test_num)
acc = (top_pred == y_test['cost']).mean()

In [ ]:
print(classification_report(y_test, top_pred))

              precision    recall  f1-score   support

           0       0.60      0.70      0.65       105
           1       0.42      0.33      0.37        70
           2       0.00      0.00      0.00         3

    accuracy                           0.54       178
   macro avg       0.34      0.34      0.34       178
weighted avg       0.52      0.54      0.53       178



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(confusion_matrix(y_test, top_pred))

[[74 31  0]
 [47 23  0]
 [ 2  1  0]]


In [ ]:
print(f'Top XGBoost Model Accuracy: {acc}')

Top XGBoost Model Accuracy: 0.5449438202247191


### GridSearch with Oversampling

In [ ]:
xgb_gridsearch_ros = XGBClassifier(objective='multi:softmax', eval_metric='merror')

In [ ]:
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.5, 1],
    'subsample': [0.6, 0.8, 1.0],
    'gamma': [0.0, 1.0],
    'colsample_bytree': [0.5, 1]
}

In [ ]:
grid_search_ros = GridSearchCV(estimator=xgb_gridsearch_ros, param_grid=param_grid,
                           scoring='accuracy', cv=5, verbose=1)

In [ ]:
grid_search_ros.fit(X_train_num_ros, y_train_ros)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False,
                                     eval_metric='merror', feature_types=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=N...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     objective='multi:softmax', ...),
             param_grid={'colsample_bytree': [0.5, 1], 'gamma': [0.0, 1.0],
                         'learning_rate': [0.1, 0.5, 1], 'max_depth': [3, 4, 5],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='accuracy', verbose=1)

In [ ]:
top_params_ros = grid_search_ros.best_params_
top_score_ros = grid_search_ros.best_score_
top_model_ros = grid_search_ros.best_estimator_

In [ ]:
top_params_ros

{'colsample_bytree': 1,
 'gamma': 0.0,
 'learning_rate': 1,
 'max_depth': 5,
 'subsample': 0.6}

In [ ]:
top_score_ros

0.726

In [ ]:
gs_oversampled_train_pred = xgb.predict(X_train_num_ros)
print(classification_report(y_train_ros, gs_oversampled_train_pred))

              precision    recall  f1-score   support

           0       0.53      0.75      0.62       500
           1       0.42      0.41      0.41       300
           2       0.00      0.00      0.00       200

    accuracy                           0.50      1000
   macro avg       0.32      0.39      0.34      1000
weighted avg       0.39      0.50      0.43      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
top_pred_ros = top_model_ros.predict(X_test_num_ros)
acc_ros = (top_pred_ros == y_test['cost']).mean()

In [ ]:
print(classification_report(y_test, top_pred_ros))

              precision    recall  f1-score   support

           0       0.60      0.61      0.60       105
           1       0.39      0.37      0.38        70
           2       0.00      0.00      0.00         3

    accuracy                           0.51       178
   macro avg       0.33      0.33      0.33       178
weighted avg       0.51      0.51      0.51       178



In [ ]:
print(confusion_matrix(y_test, top_pred_ros))

[[64 41  0]
 [40 26  4]
 [ 3  0  0]]


In [ ]:
print(f'Top XGBoost Model Accuracy: {acc_ros}')

Top XGBoost Model Accuracy: 0.5056179775280899


### SMOTE Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE(random_state=64)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

xgb = XGBClassifier(
    n_estimators=100, max_depth=2,
    learning_rate=1,
    objective='multi:softmax',
    eval_metric='mlogloss'
)

xgb.fit(X_train_smote, y_train_smote)
predictions = xgb.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.58      0.71      0.64       105
           1       0.41      0.24      0.31        70
           2       0.00      0.00      0.00         3

    accuracy                           0.52       178
   macro avg       0.33      0.32      0.31       178
weighted avg       0.50      0.52      0.50       178



In [ ]:
predictions = xgb.predict(X_train_smote)
print(classification_report(y_train_smote, predictions))

              precision    recall  f1-score   support

           0       0.78      0.89      0.83       458
           1       0.89      0.74      0.81       458
           2       0.95      0.99      0.97       458

    accuracy                           0.87      1374
   macro avg       0.87      0.87      0.87      1374
weighted avg       0.87      0.87      0.87      1374

